In [ ]:
# %load_ext tensorboard


# dir_script = ("/script")


import sys
import os

import config

import Run_qlattice

import re
import hashlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import NN_run
import prepare_data as data_source
import mongodb_driver
import requests
import feyn
import pickle
from tensorflow import keras 
from keras import layers
import utility_func as uf
import sklearn.metrics as skm 
from matplotlib import pyplot as plt
import numpy.ma as ma
from sklearn.model_selection import train_test_split
import NN_build_small
import tensorflow as tf
import kennard_stone as ks

import rfet_model as rmdl

import add_gaussian_noise
import transfer_func as transf

import matplotlib.patches as patches



In [ ]:
# maximum number of models to generate in parallel, with different seeds
i_max = 1

# PyTaurus run to extract the data from. "run_regular_grid_common_goal" is used in the dissertation
str_run         = "run_regular_grid_common_goal"
# the respective gate sweep to be used
str_electrode   = "lat2"

# minimum stepsize, refers to $\Delta V_{sweep}$ in section 4.2.1
min_step = 0.02

In [ ]:
# mdb = mongodb_driver.mdb_connect("mdb_cmodel", "felixl", "mdbpass")
mdb = mongodb_driver.mdb_connect("mdb_cmodel", "root", "t(jWbtU4DRnuXqdhK")

mdb_col = mdb["NN"]


par_offset = 8
par_slope = 4



# LINEAR MODEL

In [ ]:
res = []

for i in range(i_max):
    pars = {
        "data_type"                         : 'DC',
        "data_path"                         : f"{config.dir_data}/{str_run}_{str_electrode}_minstep{min_step}_export.pkl",
        "save_path"                         : f"NN_models/mx/lin/",

        "mdl_type"                          : 'lin',
        "mdl_arch"                          : 'NN',

        # "augmentation_type"                 : "gauss_noise_3",
        "augmentation_type"                 : None,
        "train_test_split_random_seed"      : i,
        "tf_epochs"                         : 500,
        "tf_es_patience"                    : 400,
        "tf_es_delta"                       : 1E-07,
        "tf_n_neurons"                      : 35,
        
        "user"                              : os.getlogin(),
        "comment"                           : "minimal example test"
    }

    Xlabels =   ["Vlat21", "Vfglat1", "Vtglat1"]
    Ylabel  =   ["ids"]

    ### create MongoDB document


    Xlabels =   ["Vlat21", "Vfglat1", "Vtglat1"]
    Ylabel  =   ["ids"]


    data_raw = data_source.load_dc_ramp_data(pars['data_path'], Xlabels =["Vlat21", "Vfglat1", "Vtglat1", "Vbglat1"], Ylabel=["ids"],scale_factor=1E3)

    # inv_data = data_source.load_from_DC_sim("data/diss/eval/INV/inv_n.plt",Xlabels, Ylabel, scale_factor=1 )

    # data_raw = pd.concat([inv_data, data_raw], ignore_index=True)

    data_raw_lin = data_raw[data_raw["ids"].abs() > 1e-8 ].copy()
    # data_raw_lin = data_raw.copy()
    # set current sign = vds sign
    # data_raw_lin["ids"] = data_raw_lin.apply(lambda row: abs(row["ids"])*(-1)* np.sign(row["Vlat21"]), axis=1)

    lin_prescale= 1

    data_raw_lin["ids"] = data_raw_lin["ids"].apply(lambda x: x*lin_prescale)

    trainVal = pd.DataFrame()
    test_data = pd.DataFrame()
    train_data = pd.DataFrame()
    val_data = pd.DataFrame()


    # --- creating the validation dataset for this NN train run, can be different random seed every time
    ks_x_trainVal, ks_x_test, ks_y_trainVal, ks_y_test = ks.train_test_split(data_raw_lin[Xlabels], data_raw_lin[Ylabel],test_size=0.10)

    ks_x_train, ks_x_val, ks_y_train, ks_y_val = ks.train_test_split(ks_x_trainVal, ks_y_trainVal,test_size=0.15)

    train_data[Xlabels] = ks_x_train
    train_data[Ylabel] = ks_y_train

    val_data[Xlabels] = ks_x_val
    val_data[Ylabel] = ks_y_val

    test_data[Xlabels] = ks_x_test
    test_data[Ylabel] = ks_y_test

    X = train_data.loc[:,Xlabels].to_numpy()
    y_lin = train_data.loc[:,Ylabel].to_numpy()

    X_val = val_data.loc[:,Xlabels].to_numpy()
    y_val_lin = val_data.loc[:,Ylabel].to_numpy()

    X_test = test_data.loc[:,Xlabels].to_numpy()
    y_test_lin = test_data.loc[:,Ylabel].to_numpy()

    # ----building input layer----
    x = layers.Input(shape= (3),name ="InputLayer")

    # ---- hidden layser ----
    zOne = layers.Dense(units=pars["tf_n_neurons"], activation = "tanh", name = "HiddenLayer1")(x)
    zTwo = layers.Dense(units=pars["tf_n_neurons"], activation = "tanh", name = "HiddenLayer2")(zOne)
    zThree = layers.Dense(units=pars["tf_n_neurons"], activation = "tanh", name = "HiddenLayer3")(zTwo)

    #output Layer
    y = layers.Dense(units = 1, activation = 'linear', name = "OutputLayer")(zThree)

    #build model
    mdl = keras.Model(inputs=x,outputs=y, name = "NN_model")
    print(mdl.summary())

    my_metrics = [keras.metrics.MeanAbsoluteError()]
    my_loss = keras.losses.MeanSquaredError()


    mdl.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003636),loss=my_loss,metrics = my_metrics)

    # compiled_mdl = NN_build_small.build_tanh_model(dim_in = 3, n_neurons = pars["tf_n_neurons"])

    callbacks = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=pars["tf_es_patience"],min_delta=pars["tf_es_delta"],verbose=1,mode='min',restore_best_weights=True),
            tf.keras.callbacks.TensorBoard("./NN_tuning_res/RuntimeTest_logs")]

    history = mdl.fit(X,y_lin,
                               batch_size = 100,
                               epochs = pars["tf_epochs"],
                               callbacks = callbacks,
                               validation_data=(X_val,y_val_lin),
                               verbose = 2,
                               shuffle = True)


    #----- saving the model ----
    # mdl,loss,MAE,R2, sMAPE, X, y = NN_run.mx_run_NN_lin(
    #                 save_path = pars['save_path'], 
    #                 epochs = pars['tf_epochs'],
    #                 augmentation_type = pars['augmentation_type'],
    #                 random_seed = pars["train_test_split_random_seed"],
    #                 data = data,
    #                 Xlabels = Xlabels,
    #                 Ylabel = Ylabel,
    #                 es_patience = pars["tf_es_patience"],
    #                 es_delta = pars["tf_es_delta"], 
    #                 _id = _id_lin,
    #                 # splitting_algorithm="kennard_stone",
    #                 test_size = 0.0,
    #                 n_neurons = pars["tf_n_neurons"])
    y_pred = mdl.predict(X_test,batch_size = 50)

    loss,MAE= mdl.evaluate(X_test,y_test_lin,batch_size = 100)  # returns loss and metrics

    sMAPE = uf.Smape(y_test_lin, y_pred)

    pars["lin_prescale"]                     = lin_prescale

    _id_lin = mdb_col.insert_one(pars)
    _id_lin = str(_id_lin.inserted_id)
    print(_id_lin)

    mdl.save(pars["save_path"] + "/" + str(_id_lin))

    res.append({"_id" : _id_lin, "i": i,"MAE": MAE,  "loss": loss, "model" : mdl, "sMAPE" : sMAPE})
    print(f"i: {i}\nloss {loss}\nMAE {MAE}\n")    


In [ ]:
y_pred = mdl.predict(test_data[Xlabels])

y_pred = np.array(y_pred/lin_prescale)
y_true = np.array(test_data["ids"]/lin_prescale)



In [ ]:
pd_res_lin = pd.DataFrame(res)

pd_res_lin.sort_values(by=["loss"], inplace=True, ascending=True)
pd_res_lin.reset_index(inplace=True, drop=True)

print(pd_res_lin)

# get best model
best_model_lin_id = pd_res_lin["loss"].idxmin()
best_model_lin = pd_res_lin.loc[best_model_lin_id, "model"]

_id_lin = pd_res_lin.loc[best_model_lin_id, "_id"]

In [ ]:
fig = plt.figure(figsize=(10,6))

# x_vals = test_data['time']

fig, (ax, ax2) = plt.subplots(2, sharex=True)
#fig,ax = plt.subplots()

fig.set_figwidth(10)
#ax2 = ax.twinx()

ax.set_yscale("log")
ax2.set_yscale("log")

ax2.set_xlabel("index")
#ax.set_xlabel("time")

# ax.grid()
ax2.grid()


ys = pd.DataFrame()
ys["true"] = y_true

ys["pred"] =  mdl.predict(test_data[Xlabels])/lin_prescale
# y_true_series = pd.Series(y_true)
# y_pred_series = pd.Series(y_pred)

ys.sort_values(by=["true"],inplace=True)
ys.reset_index(inplace=True)

ax.scatter(ys.index,ys["true"].abs(), label="data")
ax.scatter(ys.index,ys["pred"].abs(), label="model")

error = ys["true"] - ys["pred"]

ax2.scatter(ys.index,abs(error))

ax.legend()

In [ ]:
print(f'R2 current model: %.6f' % skm.r2_score(y_true,y_pred))
print('MAE current model: %.4E' %(skm.mean_absolute_error(y_true,y_pred)))
print('MSE current model: %.4E' %(skm.mean_squared_error(y_true,y_pred)))
print('MAPE current model: %.4E' %(skm.mean_absolute_percentage_error(y_true,y_pred)))
print('sMAPE current model: %.6f' %(uf.Smape(y_true.reshape([-1,1]),y_pred)))


In [ ]:
fig, (ax, ax2) = plt.subplots(2, sharex=True)

ax.scatter(data_raw_lin.index,data_raw_lin["ids"].abs(), label="data")


In [ ]:
inv_data = data_source.load_from_DC_sim("refs/inv_p.plt",Xlabels, Ylabel, scale_factor=1 )

inv_data.sort_values(by=["Vlat21"], inplace=True,ignore_index=True)
inv_data.reset_index(inplace=True, drop=True)

In [ ]:


fig, (ax, ax2,ax3,ax4) = plt.subplots(4, sharex=True)
#fig,ax = plt.subplots()

fig.set_figwidth(10)
fig.set_figheight(15)

#ax2 = ax.twinx()


# ax2.set_yscale("log")
ax.set_yscale("log")

#ax.set_xlabel("time")

# ax.grid()
inv_data["pred"] = mdl.predict(inv_data[Xlabels])  * 1/lin_prescale * 1/1E3

# ax.plot(inv_data.index,inv_data["pred"], label="Ilin")
ax.plot(inv_data.index,inv_data["pred"], label="Ilin")
ax.plot(inv_data.index,-inv_data["pred"], label="Ilin-")

ax.plot(inv_data.index,-inv_data["ids"], label="ref")

ax2.plot(inv_data.index,inv_data["Vlat21"])
ax3.plot(inv_data.index,inv_data["Vfglat1"])

# ax.set_ylim([1E-6, 1])
# error = abs(val_data["ids"].values.reshape(-1,1) - I)

ax.legend()

# LOGARITHMIC MODEL

In [ ]:
res = []

for i in range(i_max):
    pars = {
        "data_type"                         : 'DC',
        "data_path"                         : f"{config.dir_data}/{str_run}_{str_electrode}_minstep{min_step}_export.pkl",
        "save_path"                         : f"NN_models/mx/log/",

        "mdl_type"                          : 'log',
        "mdl_arch"                          : 'NN',

        # "augmentation_type"                 : "gauss_noise_3",
        "augmentation_type"                 : None,
        "train_test_split_random_seed"      : i,
        "tf_epochs"                         : 500,
        "tf_es_patience"                    : 400,
        "tf_es_delta"                       : 1E-07,
        "tf_n_neurons"                      : 34,
        
        "user"                              : os.getlogin(),
        "comment"                           : "minimal example test"
    }

    Xlabels =   ["Vlat21", "Vfglat1", "Vtglat1"]
    Ylabel  =   ["ids"]

    ### create MongoDB document


    Xlabels =   ["Vlat21", "Vfglat1", "Vtglat1"]
    Ylabel  =   ["ids"]


    data_raw = data_source.load_dc_ramp_data(pars['data_path'], Xlabels =["Vlat21", "Vfglat1", "Vtglat1", "Vbglat1"], Ylabel=["ids"],scale_factor=1E3)

    # inv_data = data_source.load_from_DC_sim("data/diss/eval/INV/inv_n.plt",Xlabels, Ylabel, scale_factor=1 )

    # data_raw = pd.concat([inv_data, data_raw], ignore_index=True)

    # data_raw_lin = data_raw[data_raw["ids"].abs() > 1e-7 ].copy()
    # data_raw_lin = data_raw.copy()
    # set current sign = vds sign
    # data_raw_lin["ids"] = data_raw_lin.apply(lambda row: abs(row["ids"])*(-1)* np.sign(row["Vlat21"]), axis=1)

    # lin_prescale= 1/data_raw_lin["ids"].abs().max()

    # data_raw_lin["ids"] = data_raw_lin["ids"].apply(lambda x: x*lin_prescale)

    trainVal = pd.DataFrame()
    test_data = pd.DataFrame()
    train_data = pd.DataFrame()
    val_data = pd.DataFrame()


    # --- creating the validation dataset for this NN train run, can be different random seed every time
    ks_x_trainVal, ks_x_test, ks_y_trainVal, ks_y_test = ks.train_test_split(data_raw[Xlabels], data_raw[Ylabel],test_size=0.10)

    ks_x_train, ks_x_val, ks_y_train, ks_y_val = ks.train_test_split(ks_x_trainVal, ks_y_trainVal,test_size=0.15)

    train_data[Xlabels] = ks_x_train
    train_data[Ylabel] = ks_y_train

    val_data[Xlabels] = ks_x_val
    val_data[Ylabel] = ks_y_val

    test_data[Xlabels] = ks_x_test
    test_data[Ylabel] = ks_y_test

    X = train_data.loc[:,Xlabels].to_numpy()
    y= train_data.loc[:,Ylabel].to_numpy()

    X_val = val_data.loc[:,Xlabels].to_numpy()
    y_val = val_data.loc[:,Ylabel].to_numpy()

    X_test = test_data.loc[:,Xlabels].to_numpy()
    y_test = test_data.loc[:,Ylabel].to_numpy()

    Xy_train = pd.DataFrame(X.copy())
    # Xy_train.columns = ["Vlat21", "Vfglat1", "Vtglat1", "Vbglat1"]
    Xy_train.rename(columns={0:"Vlat21", 1: "Vfglat1", 2: "Vtglat1"}, inplace=True)
    Xy_train["ids"] = y

    Xy_val = pd.DataFrame(X_val.copy())
    Xy_val.rename(columns={0:"Vlat21", 1: "Vfglat1", 2: "Vtglat1"}, inplace=True)
    Xy_val["ids"] = y_val

    Xy_test = pd.DataFrame(X_test.copy())
    Xy_test.rename(columns={0:"Vlat21", 1: "Vfglat1", 2: "Vtglat1"}, inplace=True)
    Xy_test["ids"] = y_test

    Xy_train_aug = add_gaussian_noise.augment_train_with_noise(trainVal = Xy_train, num_copies = 3 ,yname=Ylabel[0],sigma=1e-12)    

    Xy_train_aug.sort_values(by=["ids"], inplace=True, ignore_index=True)
    Xy_train_aug.reset_index(inplace=True, drop=True)

    # Xy_train_aug.columns = ["Vlat21", "Vfglat1", "Vtglat1", "Vbglat1", "ids"]
    Xy_train_aug.rename(columns={0:"Vlat21", 1: "Vfglat1", 2: "Vtglat1", 3: "ids"}, inplace=True)

    Xy_train_aug_log,_ = transf.transf_Thung(data = Xy_train_aug, Vds="Vlat21", yname = Ylabel[0], improved_del='on', epsilon=1E-3)
    Xy_val_log,_ = transf.transf_Thung(data = Xy_val, Vds="Vlat21", yname = Ylabel[0])
    Xy_test_log,_ = transf.transf_Thung(data = Xy_test, Vds="Vlat21", yname = Ylabel[0])



    # ----building input layer----
    x = layers.Input(shape= (3),name ="InputLayer")

    # ---- hidden layser ----
    zOne = layers.Dense(units=pars["tf_n_neurons"], activation = "tanh", name = "HiddenLayer1")(x)
    zTwo = layers.Dense(units=pars["tf_n_neurons"], activation = "tanh", name = "HiddenLayer2")(zOne)
    zThree = layers.Dense(units=pars["tf_n_neurons"], activation = "tanh", name = "HiddenLayer3")(zTwo)

    #output Layer
    y = layers.Dense(units = 1, activation = 'linear', name = "OutputLayer")(zThree)
    
    def rmse_loss(y_true, y_pred):
        return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))
    
    #build model
    mdl = keras.Model(inputs=x,outputs=y, name = "NN_model")
    print(mdl.summary())

    my_metrics = [keras.metrics.MeanAbsoluteError()]
    my_loss = keras.losses.MeanSquaredError()
    # my_loss = keras.losses.MeanAbsoluteError()


    mdl.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001113),loss=my_loss,metrics = my_metrics)

    # compiled_mdl = NN_build_small.build_tanh_model(dim_in = 3, n_neurons = pars["tf_n_neurons"])

    callbacks = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=pars["tf_es_patience"],min_delta=pars["tf_es_delta"],verbose=1,mode='min',restore_best_weights=True),
            tf.keras.callbacks.TensorBoard("./NN_tuning_res/RuntimeTest_logs")]


    history = mdl.fit(Xy_train_aug_log[Xlabels],Xy_train_aug_log["y_tilde"],
                               batch_size = 100,
                               epochs = pars["tf_epochs"],
                               callbacks = callbacks,
                               validation_data=(Xy_val_log[Xlabels],Xy_val_log["y_tilde"]),
                               verbose = 2,
                               shuffle = True)


    #----- saving the model ----
    # mdl,loss,MAE,R2, sMAPE, X, y = NN_run.mx_run_NN_lin(
    #                 save_path = pars['save_path'], 
    #                 epochs = pars['tf_epochs'],
    #                 augmentation_type = pars['augmentation_type'],
    #                 random_seed = pars["train_test_split_random_seed"],
    #                 data = data,
    #                 Xlabels = Xlabels,
    #                 Ylabel = Ylabel,
    #                 es_patience = pars["tf_es_patience"],
    #                 es_delta = pars["tf_es_delta"], 
    #                 _id = _id_lin,
    #                 # splitting_algorithm="kennard_stone",
    #                 test_size = 0.0,
    #                 n_neurons = pars["tf_n_neurons"])
    y_pred = mdl.predict(Xy_test_log[Xlabels],batch_size = 50)

    loss,MAE= mdl.evaluate(Xy_test_log[Xlabels],Xy_test_log["y_tilde"],batch_size = 100)  # returns loss and metrics

    sMAPE = uf.Smape(Xy_test_log["y_tilde"].to_numpy().reshape([-1,1]), y_pred)

    # pars["lin_prescale"]                     = lin_prescale

    _id_log = mdb_col.insert_one(pars)
    _id_log = str(_id_log.inserted_id)
    print(_id_log)

    mdl.save(pars["save_path"] + "/" + str(_id_log))

    res.append({"_id" : _id_log, "i": i,"MAE": MAE,  "loss": loss, "model" : mdl, "sMAPE" : sMAPE})
    print(f"i: {i}\nloss {loss}\nMAE {MAE}\n")    


In [ ]:
y_pred = mdl.predict(Xy_test_log[Xlabels])

y_pred = np.array(y_pred)
y_true = np.array(Xy_test_log["y_tilde"])



In [ ]:
pd_res_log = pd.DataFrame(res)

pd_res_log.sort_values(by=["loss"], inplace=True, ascending=True)
pd_res_log.reset_index(inplace=True, drop=True)

print(pd_res_log)

# get best model
best_model_log_id = pd_res_log["loss"].idxmin()
best_model_log = pd_res_log.loc[best_model_log_id, "model"]

_id_log = pd_res_log.loc[best_model_log_id, "_id"]

In [ ]:
fig = plt.figure(figsize=(10,6))

# x_vals = test_data['time']

fig, (ax, ax2) = plt.subplots(2, sharex=True)
#fig,ax = plt.subplots()

fig.set_figwidth(10)
#ax2 = ax.twinx()

ax.set_yscale("log")
ax2.set_yscale("log")

ax2.set_xlabel("index")
#ax.set_xlabel("time")

# ax.grid()
ax2.grid()


ys = pd.DataFrame()
ys["true"] = y_true

ys["pred"] =  mdl.predict(Xy_test_log[Xlabels])
# y_true_series = pd.Series(y_true)
# y_pred_series = pd.Series(y_pred)

ys.sort_values(by=["true"],inplace=True)
ys.reset_index(inplace=True)

ax.scatter(ys.index,ys["true"].abs(), label="data")
ax.scatter(ys.index,ys["pred"].abs(), label="model")

error = ys["true"] - ys["pred"]

ax2.scatter(ys.index,abs(error))

ax.legend()

In [ ]:
print(f'R2 current model: %.6f' % skm.r2_score(y_true,y_pred))
print('MAE current model: %.4E' %(skm.mean_absolute_error(y_true,y_pred)))
print('MSE current model: %.4E' %(skm.mean_squared_error(y_true,y_pred)))
print('MAPE current model: %.4E' %(skm.mean_absolute_percentage_error(y_true,y_pred)))
print('sMAPE current model: %.6f' %(uf.Smape(y_true.reshape([-1,1]),y_pred)))


In [ ]:
fig, (ax, ax2) = plt.subplots(2, sharex=True)

ax.scatter(data_raw.index,data_raw["ids"].abs(), label="data")


In [ ]:
inv_data = data_source.load_from_DC_sim("refs/inv_p.plt",Xlabels, Ylabel, scale_factor=1 )

inv_data.sort_values(by=["Vlat21"], inplace=True,ignore_index=True)
inv_data.reset_index(inplace=True, drop=True)

In [ ]:


fig, (ax, ax2,ax3,ax4) = plt.subplots(4, sharex=True)
#fig,ax = plt.subplots()

fig.set_figwidth(10)
fig.set_figheight(15)

#ax2 = ax.twinx()


# ax2.set_yscale("log")
ax.set_yscale("log")

#ax.set_xlabel("time")

# ax.grid()
inv_data["pred"] = mdl.predict(inv_data[Xlabels])

inv_data["pred"] = -np.multiply(inv_data["Vlat21"], np.exp(inv_data["pred"]) ) * 1E-3


# ax.plot(inv_data.index,inv_data["pred"], label="Ilin")
ax.plot(inv_data.index,inv_data["pred"], label="Ilin")
ax.plot(inv_data.index,-inv_data["pred"], label="Ilin-")

ax.plot(inv_data.index,abs(inv_data["ids"]), label="ref")

ax2.plot(inv_data.index,inv_data["Vlat21"])
ax3.plot(inv_data.index,inv_data["Vfglat1"])

# ax.set_ylim([1E-6, 1])
# error = abs(val_data["ids"].values.reshape(-1,1) - I)

ax.legend()

In [ ]:
inv_data = data_source.load_from_DC_sim("refs/inv_p.plt",Xlabels, Ylabel, scale_factor=1 )

inv_data.sort_values(by=["Vlat21"], inplace=True,ignore_index=True)
inv_data.reset_index(inplace=True, drop=True)

In [ ]:

fig, (ax, ax2,ax3,ax4) = plt.subplots(4, sharex=True)
#fig,ax = plt.subplots()

fig.set_figwidth(10)
fig.set_figheight(15)

#ax2 = ax.twinx()


# ax2.set_yscale("log")
ax.set_yscale("log")

#ax.set_xlabel("time")

# ax.grid()
inv_data["pred"] = mdl.predict(inv_data[Xlabels])

inv_data["pred"] = -np.multiply(inv_data["Vlat21"], np.exp(inv_data["pred"]) ) *1E-3


# ax.plot(inv_data.index,inv_data["pred"], label="Ilin")
ax.plot(inv_data.index,inv_data["pred"], label="Ilin")
ax.plot(inv_data.index,-inv_data["pred"], label="Ilin-")

ax.plot(inv_data.index,abs(inv_data["ids"]), label="ref")

ax2.plot(inv_data.index,inv_data["Vlat21"])
ax3.plot(inv_data.index,inv_data["Vfglat1"])

# ax.set_ylim([1E-6, 1])
# error = abs(val_data["ids"].values.reshape(-1,1) - I)

ax.legend()

# BUILD MODEL

In [ ]:
_id_lin

In [ ]:
mdl_nn = rmdl.rfet_model("testname", template_file= "templates/verilogA_template_par_A.va", I_model = {"lin" : _id_lin, "log" : _id_log}, Q_model = "668be7cf9144e1dfe0885701")     # 16 neurons, Q model aus DC Daten

mdl_nn.setEnsemblePars(8 , 4) # check hyperparameters in diss
mdl_nn.compile(path="export/veriloga.va", significant_digits = 12)